# Codigo Extracción

In [ ]:
import requests
import pandas as pd
from google.cloud import storage

def extract_wb_data(request):
    # Definir los países, indicadores y años
    countries = ["ARG", "BOL", "BRA", "CAN", "CHL", "COL", "CRI", "CUB", "DOM", "ECU", "SLV", "GTM", "HTI", "HND", "JAM", "MEX", "NIC", "PAN", "PRY", "PER", "USA", "URY", "VEN"]
    indicators = ['EN.ATM.CO2E.PP.GD', 'SP.DYN.LE00.IN', 'SH.H2O.SMDW.ZS', 'SP.RUR.TOTL.ZS', 'SP.DYN.CDRT.IN','SH.XPD.GHED.GD.ZS','SP.DYN.LE00.FE.IN','SE.ADT.LITR.ZS','SP.DYN.AMRT.FE', 'SP.DYN.AMRT.MA', 'SP.DYN.IMRT.IN','NY.GDP.PCAP.PP.KD', 'SP.DYN.CBRT.IN']
    start_year = "1990"
    end_year = "2021"

    # Lista para almacenar los datos
    data_frames = []

    # URL base de la API del Banco Mundial
    base_url = "http://api.worldbank.org/v2/country"

    # Realizar las consultas para cada país, indicador y año
    for country_code in countries:
        for indicator in indicators:
            # Construir la URL de la consulta
            url = f"{base_url}/{country_code}/indicator/{indicator}?date={start_year}:{end_year}&format=json"

            # Realizar la solicitud GET a la API del Banco Mundial
            response = requests.get(url)

            # Verificar si la solicitud fue exitosa
            if response.status_code == 200:
                data = response.json()

                if data[1]:
                # Los datos se encuentran en data[1]
                    for entry in data[1]:
                        year = entry['date']
                        value = entry['value']
                        indicator_name = entry['indicator']['value']
                        country_name = entry['country']['value']
                        data_frames.append(pd.DataFrame({"Pais": [country_name], "Indicador": [indicator_name], "Anio": [year], "Valor": [value]}))
                else:
                    print(f"No hay datos disponibles para {country_code} y {indicator}.")
            else:
                print(f"Error al obtener datos para {country_code} y {indicator}. Código de estado: {response.status_code}")

    # Concatenar todos los DataFrames individuales en uno
    data_df = pd.concat(data_frames, ignore_index=True)

    # Exportar el DataFrame a un archivo CSV
    csv_data = data_df.to_csv(index=False)

    # Guardar el archivo CSV en Google Cloud Storage
    bucket_name = "bucket-extraction"  # Reemplaza con el nombre de tu bucket
    file_name = "wb_data.csv"  # Nombre del archivo CSV en el bucket

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    try:
        blob.upload_from_string(csv_data, content_type="text/csv")
        return "Datos extraídos y almacenados en Google Cloud Storage."
    except Exception as e:
        return f"Error al cargar el archivo en Google Cloud Storage: {str(e)}"

    #blob.upload_from_string(csv_data, content_type="text/csv")

    return "Datos extraídos y almacenados en Google Cloud Storage."

#requests
#pandas
#google-cloud-storage

# Codigo transformación

In [ ]:
import pandas as pd
from flask import jsonify, abort, Request
from google.cloud import storage
import io
import numpy as np
from sklearn.linear_model import LinearRegression

def mark_outliers_with_nan(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1 - 1.5 * IQR
    upper_whisker = Q3 + 1.5 * IQR
    
    outliers_mask = ((df[column] < lower_whisker) | (df[column] > upper_whisker))
    df.loc[outliers_mask, column] = np.nan
    return df

def imputation_for_all_indicators(df):
    for column in df.columns[2:]:
        df = linear_regression_imputation(df, column)
    return df

def linear_regression_imputation(df, column):
    df_subset = df[df[column].notna()]
    if len(df_subset) > 2:
        X_train = df_subset[['Anio']].values.reshape(-1, 1)
        y_train = df_subset[column].values

        model = LinearRegression()
        model.fit(X_train, y_train)

        df_missing = df[df[column].isna()]
        if not df_missing.empty:
            X_missing = df_missing[['Anio']].values.reshape(-1, 1)
            predictions = model.predict(X_missing)
            df.loc[df_missing.index, column] = predictions
    return df

def process_csv(request: Request):
    try:
        # Configura el cliente de Google Cloud Storage
        storage_client = storage.Client()

        # Especifica el nombre del bucket y el archivo CSV
        bucket_name = 'bucket-extraction'
        file_name = 'wb_data.csv'

        # Obtiene el bucket y el archivo
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(file_name)

        # Descarga el archivo CSV a un objeto BytesIO
        csv_data = blob.download_as_text()

        # Crea un DataFrame a partir del CSV
        df = pd.read_csv(io.StringIO(csv_data))

        # Transformaciones adicionales
        pivoted_df = df.pivot(index=['Pais', 'Anio'], columns='Indicador', values='Valor').reset_index()
        pivoted_df.drop_duplicates(inplace=True)
        pivoted_df.replace(0, pd.NA, inplace=True)

        # Detección y manejo de outliers
        for column in pivoted_df.select_dtypes(include=np.number).columns:
            pivoted_df = mark_outliers_with_nan(pivoted_df, column)

        # Imputación de valores nulos
        pivoted_df = imputation_for_all_indicators(pivoted_df)

        # Guardar el DataFrame resultante como un nuevo archivo CSV en el bucket
        new_file_name = 'processed_data.csv'
        new_blob = bucket.blob(new_file_name)
        csv_content = pivoted_df.to_csv(index=False)
        new_blob.upload_from_string(csv_content)

        return jsonify({"success": True})

    except Exception as e:
        print(f"Error: {e}")
        abort(500, description=str(e))

#google-cloud-storage==2.13.0
#pandas==2.1.3
#numpy==1.26.1
#scikit-learn==0.24.2  # Specify the version of scikit-learn

## Transformación tercer sprint

In [ ]:
import pandas as pd
from flask import jsonify, abort, Request
from google.cloud import storage
import io
import numpy as np
from sklearn.linear_model import LinearRegression

def mark_outliers_with_nan(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1 - 1.5 * IQR
    upper_whisker = Q3 + 1.5 * IQR
    
    outliers_mask = ((df[column] < lower_whisker) | (df[column] > upper_whisker))
    df.loc[outliers_mask, column] = np.nan
    return df

def imputation_for_all_indicators(df):
    for column in df.columns[2:]:
        df = linear_regression_imputation(df, column)
    return df

def linear_regression_imputation(df, column):
    df_subset = df[df[column].notna()]
    if len(df_subset) > 2:
        X_train = df_subset[['Anio']].values.reshape(-1, 1)
        y_train = df_subset[column].values

        model = LinearRegression()
        model.fit(X_train, y_train)

        df_missing = df[df[column].isna()]
        if not df_missing.empty:
            X_missing = df_missing[['Anio']].values.reshape(-1, 1)
            predictions = model.predict(X_missing)
            df.loc[df_missing.index, column] = predictions
    return df

def process_csv(request: Request):
    try:
        # Configura el cliente de Google Cloud Storage
        storage_client = storage.Client()

        # Especifica el nombre del bucket y el archivo CSV
        bucket_name = 'bucket-extraction'
        file_name = 'wb_data_machine_learning.csv'

        # Obtiene el bucket y el archivo
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(file_name)

        # Descarga el archivo CSV a un objeto BytesIO
        csv_data = blob.download_as_text()

        # Crea un DataFrame a partir del CSV
        df = pd.read_csv(io.StringIO(csv_data))

        # Transformaciones adicionales
        pivoted_df = df.pivot(index=['Pais', 'Anio'], columns='Indicador', values='Valor').reset_index()
        pivoted_df.drop_duplicates(inplace=True)
        pivoted_df.replace(0, pd.NA, inplace=True)

        # Detección y manejo de outliers
        for column in pivoted_df.select_dtypes(include=np.number).columns:
            pivoted_df = mark_outliers_with_nan(pivoted_df, column)

        # Imputación de valores nulos
        pivoted_df = imputation_for_all_indicators(pivoted_df)

        # Renombrar columnas
        pivoted_df.rename(columns={'Pais': 'Pais',
                    'Anio': 'Anio',
                    'Birth rate, crude (per 1,000 people)': 'Tasa_Natalidad',
                    'CO2 emissions (kg per PPP $ of GDP)': 'Emisiones_CO2',
                    'Compulsory education, duration (years)':'Educacion_obligatoria_en_anios',
                    'Death rate, crude (per 1,000 people)': 'Tasa_Mortalidad',
                    'Domestic general government health expenditure (% of GDP)': 'Gasto_Salud_Gobierno',
                    'GDP per capita, PPP (constant 2017 international $)': 'PIB_per_capita',
                    'Life expectancy at birth, female (years)': 'Esperanza_vida_femenina',
                    'Life expectancy at birth, total (years)': 'Esperanza_vida_total',
                    'Literacy rate, adult total (% of people ages 15 and above)': 'Tasa_alfabetizacion_adultos',
                    'Mortality rate, adult, female (per 1,000 female adults)': 'Mortalidad_adulta_femenina',
                    'Mortality rate, adult, male (per 1,000 male adults)': 'Mortalidad_adulta_masculina',
                    'Mortality rate, infant (per 1,000 live births)': 'Mortalidad_infantil',
                    'People using safely managed drinking water services (% of population)': 'Acceso_agua_potable',
                    'Rural population (% of total population)': 'Poblacion_rural',

                    'Current health expenditure per capita, PPP (current international $)': 'gasto_salud_per_capita_ppp',
                    'Domestic general government health expenditure per capita (current US$)': 'gasto_salud_gobierno_per_capita_ppp',
                    'Domestic private health expenditure (% of current health expenditure)': 'gasto_salud_privado_pct_gasto_salud_actual',
                    'Educational attainment, at least completed lower secondary, population 25+, total (%) (cumulative)': 'logro_educativo_secundaria_inferior_pct_poblacion_25_anios_mas',
                    'Gini index': 'indice_gini',
                    'Government expenditure on education, total (% of GDP)': 'gasto_educacion_gobierno_pct_pib',
                    'Mortality caused by road traffic injury (per 100,000 population)': 'tasa_mortalidad_lesiones_trafico',
                    'Mortality from CVD, cancer, diabetes or CRD between exact ages 30 and 70 (%)': 'mortalidad_enfermedades',
                    'People using at least basic sanitation services (% of population)': 'personas_saneamiento_basico_pct_poblacion',
                    'PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)': 'contaminacion_pct_poblacion_excede_oms',
                    'Political Stability and Absence of Violence/Terrorism: Estimate': 'estabilidad_politica',
                    'Poverty gap at $2.15 a day (2017 PPP) (%)': 'brecha_pobreza_2_15_dolars_a_day',
                    'Prevalence of undernourishment (% of population)': 'prevalencia_desnutricion_pct_poblacion',
                    'Unemployment, total (% of total labor force) (modeled ILO estimate)': 'desempleo_total_ilo',
                    'Urban population': 'poblacion_urbana'}, inplace=True)

        pivoted_df['Pais'] = pivoted_df['Pais'].replace('Venezuela, RB', 'Venezuela')
        
        # Guardar el DataFrame resultante como un nuevo archivo CSV en el bucket
        new_file_name = 'processed_data_machine_learning.csv'
        new_blob = bucket.blob(new_file_name)
        csv_content = pivoted_df.to_csv(index=False)
        new_blob.upload_from_string(csv_content)

        return jsonify({"success": True})

    except Exception as e:
        print(f"Error: {e}")
        abort(500, description=str(e))

# Codigo trigger

In [ ]:
import requests
import pandas as pd
from google.cloud import storage

def trigger_wb_data(request):
    # Definir los países, indicadores y años
    countries = ["ARG", "BOL", "BRA", "CAN", "CHL", "COL", "CRI", "CUB", "DOM", "ECU", "SLV", "GTM", "HTI", "HND", "JAM", "MEX", "NIC", "PAN", "PRY", "PER", "USA", "URY", "VEN"]
    indicators = ['EN.ATM.CO2E.PP.GD', 'SP.DYN.LE00.IN', 'SH.H2O.SMDW.ZS', 'SP.RUR.TOTL.ZS', 'SP.DYN.CDRT.IN','SH.XPD.GHED.GD.ZS','SP.DYN.LE00.FE.IN','SE.ADT.LITR.ZS','SP.DYN.AMRT.FE', 'SP.DYN.AMRT.MA', 'SP.DYN.IMRT.IN','NY.GDP.PCAP.PP.KD', 'SP.DYN.CBRT.IN']
    start_year = "2022"

    # Lista para almacenar los datos
    data_frames = []

    # URL base de la API del Banco Mundial
    base_url = "http://api.worldbank.org/v2/country"

    # Realizar las consultas para cada país, indicador y año
    for country_code in countries:
        for indicator in indicators:
            # Construir la URL de la consulta
            url = f"{base_url}/{country_code}/indicator/{indicator}?date={start_year}&format=json"

            # Realizar la solicitud GET a la API del Banco Mundial
            response = requests.get(url)

            # Verificar si la solicitud fue exitosa
            if response.status_code == 200:
                data = response.json()

                if data[1]:
                # Los datos se encuentran en data[1]
                    for entry in data[1]:
                        year = entry['date']
                        value = entry['value']
                        indicator_name = entry['indicator']['value']
                        country_name = entry['country']['value']
                        data_frames.append(pd.DataFrame({"Pais": [country_name], "Indicador": [indicator_name], "Anio": [year], "Valor": [value]}))
                else:
                    print(f"No hay datos disponibles para {country_code} y {indicator}.")
            else:
                print(f"Error al obtener datos para {country_code} y {indicator}. Código de estado: {response.status_code}")

    # Concatenar todos los DataFrames individuales en uno
    data_df = pd.concat(data_frames, ignore_index=True)

    # Convertir DataFrame a formato CSV en memoria sin incluir encabezados ni índices
    csv_content = data_df.to_csv(index=False, header=False)

    # Configurar el cliente de Cloud Storage
    client = storage.Client()

    # Especificar el nombre del archivo CSV en tu bucket
    bucket_name = 'bucket-extraction'
    file_name = 'wb_data.csv'
    
    # Obtener el bucket
    bucket = client.get_bucket(bucket_name)

    # Verificar si el archivo ya existe en el bucket
    blob = bucket.blob(file_name)

    if blob.exists():
        # Descargar el contenido existente del archivo CSV en el bucket
        existing_content = blob.download_as_text()

        # Verificar si la información del año 2022 ya está en el archivo existente
        if '2022' not in existing_content.split('\n'):
            # Agregar una línea vacía si el archivo existente no termina con una línea vacía
            if existing_content and not existing_content.endswith('\n'):
                existing_content += '\n'
            
            # Concatenar el nuevo contenido con el contenido existente
            csv_content = existing_content + csv_content

            # Cargar el contenido actualizado en el archivo CSV en el bucket
            blob.upload_from_string(csv_content, content_type='text/csv')

            print(f"Información del año 2022 añadida a {bucket_name}/{file_name}")
        else:
            print(f"Información del año 2022 ya existe en {bucket_name}/{file_name}. No se ha añadido nada.")
    else:
        # Si el archivo no existe, cargar el contenido del DataFrame como nuevo archivo
        blob.upload_from_string(csv_content, content_type='text/csv')

        print(f"Nuevo archivo CSV creado en {bucket_name}/{file_name}")

    return "Proceso completado exitosamente"

#requests
#pandas
#google-cloud-storage
#fsspec
#gcsfs

In [ ]:
import pandas as pd
from flask import jsonify, abort, Request
from google.cloud import storage
import io
import numpy as np
from sklearn.linear_model import LinearRegression

def mark_outliers_with_nan(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1 - 1.5 * IQR
    upper_whisker = Q3 + 1.5 * IQR
    
    outliers_mask = ((df[column] < lower_whisker) | (df[column] > upper_whisker))
    df.loc[outliers_mask, column] = np.nan
    return df

def imputation_for_all_indicators(df):
    for column in df.columns[2:]:
        df = linear_regression_imputation(df, column)
    return df

def linear_regression_imputation(df, column):
    df_subset = df[df[column].notna()]
    if len(df_subset) > 2:
        X_train = df_subset[['Anio']].values.reshape(-1, 1)
        y_train = df_subset[column].values

        model = LinearRegression()
        model.fit(X_train, y_train)

        df_missing = df[df[column].isna()]
        if not df_missing.empty:
            X_missing = df_missing[['Anio']].values.reshape(-1, 1)
            predictions = model.predict(X_missing)
            df.loc[df_missing.index, column] = predictions
    return df

def process_csv(request: Request):
    try:
        # Configura el cliente de Google Cloud Storage
        storage_client = storage.Client()

        # Especifica el nombre del bucket y el archivo CSV
        bucket_name = 'bucket-extraction'
        file_name = 'wb_data.csv'

        # Obtiene el bucket y el archivo
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(file_name)

        # Descarga el archivo CSV a un objeto BytesIO
        csv_data = blob.download_as_text()

        # Crea un DataFrame a partir del CSV
        df = pd.read_csv(io.StringIO(csv_data))

        # Transformaciones adicionales
        pivoted_df = df.pivot(index=['Pais', 'Anio'], columns='Indicador', values='Valor').reset_index()
        pivoted_df.drop_duplicates(inplace=True)
        pivoted_df.replace(0, pd.NA, inplace=True)

        # Detección y manejo de outliers
        for column in pivoted_df.select_dtypes(include=np.number).columns:
            pivoted_df = mark_outliers_with_nan(pivoted_df, column)

        # Imputación de valores nulos
        pivoted_df = imputation_for_all_indicators(pivoted_df)

        # Renombrar columnas
        pivoted_df.rename(columns={'Pais': 'Pais',
                    'Anio': 'Anio',
                    'Birth rate, crude (per 1,000 people)': 'Tasa_Natalidad',
                    'CO2 emissions (kg per PPP $ of GDP)': 'Emisiones_CO2',
                    'Compulsory education, duration (years)':'Educacion_obligatoria_en_años',
                    'Death rate, crude (per 1,000 people)': 'Tasa_Mortalidad',
                    'Domestic general government health expenditure (% of GDP)': 'Gasto_Salud_Gobierno',
                    'GDP per capita, PPP (constant 2017 international $)': 'PIB_per_capita',
                    'Life expectancy at birth, female (years)': 'Esperanza_vida_femenina',
                    'Life expectancy at birth, total (years)': 'Esperanza_vida_total',
                    'Literacy rate, adult total (% of people ages 15 and above)': 'Tasa_alfabetización_adultos',
                    'Mortality rate, adult, female (per 1,000 female adults)': 'Mortalidad_adulta_femenina',
                    'Mortality rate, adult, male (per 1,000 male adults)': 'Mortalidad_adulta_masculina',
                    'Mortality rate, infant (per 1,000 live births)': 'Mortalidad_infantil',
                    'People using safely managed drinking water services (% of population)': 'Acceso_agua_potable',
                    'Rural population (% of total population)': 'Población_rural'}, inplace=True)

        # Cambiar el tipo de dato de la columna "nombre_columna" a entero
        pivoted_df['Anio'] = pivoted_df['Anio'].astype(str)

        # Guardar el DataFrame resultante como un nuevo archivo CSV en el bucket
        new_file_name = 'processed_data.csv'
        new_blob = bucket.blob(new_file_name)
        csv_content = pivoted_df.to_csv(index=False)
        new_blob.upload_from_string(csv_content)

        return jsonify({"success": True})

    except Exception as e:
        print(f"Error: {e}")
        abort(500, description=str(e)) 

In [ ]:
import os
from google.cloud import storage
from google.cloud import bigquery
from google.cloud.exceptions import NotFound

def process_file(event, context):
    # Extraer el nombre del archivo del evento de Cloud Storage
    file_name = event['processed_data.csv']
    bucket_name = event['bucket-extraction']

    # Configurar el cliente de Google Cloud Storage
    storage_client = storage.Client()

    # Descargar el archivo desde Cloud Storage
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    file_content = blob.download_as_text()

    # Configurar el cliente de BigQuery
    bq_client = bigquery.Client()

    # Configurar el nombre de tu conjunto de datos y tabla en BigQuery
    dataset_id = 'bucket_extraction_wb_data'
    table_id = 'tabla_transformada_prueba'

    # Crear una tabla si no existe
    dataset_ref = bq_client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    
    try:
        bq_client.get_table(table_ref)
    except NotFound:
        # Crear la tabla si no existe
        schema = [
            bigquery.SchemaField('Pais', 'STRING'),
            bigquery.SchemaField('Anio', 'INTEGER'),
            bigquery.SchemaField('Tasa_Natalidad', 'FLOAT'),
            bigquery.SchemaField('Emisiones_CO2', 'FLOAT'),
            bigquery.SchemaField('Educacion_obligatoria_en_a__os', 'FLOAT'),
            bigquery.SchemaField('Tasa_Mortalidad', 'FLOAT'),
            bigquery.SchemaField('Gasto_Salud_Gobierno', 'FLOAT'),
            bigquery.SchemaField('PIB_per_capita', 'FLOAT'),
            bigquery.SchemaField('Esperanza_vida_femenina', 'FLOAT'),
            bigquery.SchemaField('Esperanza_vida_total', 'FLOAT'),
            bigquery.SchemaField('Tasa_alfabetizaci__n_adultos', 'FLOAT'),
            bigquery.SchemaField('Mortalidad_adulta_femenina', 'FLOAT'),
            bigquery.SchemaField('Mortalidad_adulta_masculina', 'FLOAT'),
            bigquery.SchemaField('Mortalidad_infantil', 'FLOAT'),
            bigquery.SchemaField('Acceso_agua_potable', 'FLOAT'),
            bigquery.SchemaField('Poblaci__n_rural', 'FLOAT'),
            # Agrega más campos según tu esquema
        ]
        table = bigquery.Table(table_ref, schema=schema)
        bq_client.create_table(table)

    # Cargar datos en BigQuery
    job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField('Pais', 'STRING'),
            bigquery.SchemaField('Anio', 'INTEGER'),
            bigquery.SchemaField('Tasa_Natalidad', 'FLOAT'),
            bigquery.SchemaField('Emisiones_CO2', 'FLOAT'),
            bigquery.SchemaField('Educacion_obligatoria_en_a__os', 'FLOAT'),
            bigquery.SchemaField('Tasa_Mortalidad', 'FLOAT'),
            bigquery.SchemaField('Gasto_Salud_Gobierno', 'FLOAT'),
            bigquery.SchemaField('PIB_per_capita', 'FLOAT'),
            bigquery.SchemaField('Esperanza_vida_femenina', 'FLOAT'),
            bigquery.SchemaField('Esperanza_vida_total', 'FLOAT'),
            bigquery.SchemaField('Tasa_alfabetizaci__n_adultos', 'FLOAT'),
            bigquery.SchemaField('Mortalidad_adulta_femenina', 'FLOAT'),
            bigquery.SchemaField('Mortalidad_adulta_masculina', 'FLOAT'),
            bigquery.SchemaField('Mortalidad_infantil', 'FLOAT'),
            bigquery.SchemaField('Acceso_agua_potable', 'FLOAT'),
            bigquery.SchemaField('Poblaci__n_rural', 'FLOAT'),
            # Agrega más campos según tu esquema
        ],
        skip_leading_rows=1,
        source_format=bigquery.SourceFormat.CSV,
    )

    job = bq_client.load_table_from_file(file_content, table_ref, job_config=job_config)
    job.result()

    print(f"Datos cargados exitosamente en {dataset_id}.{table_id}")

    return "Proceso completado con éxito."

In [ ]:
from google.cloud import bigquery
from google.cloud import storage

def cargar_datos(request):
    # Inicializa BigQuery y Storage
    client = bigquery.Client()
    storage_client = storage.Client()

    # Configuración del archivo y el bucket
    bucket_name = 'bucket-extraction'
    file_name = 'processed_data.csv'

    # Configuración del conjunto de datos y la tabla en BigQuery
    dataset_id = 'bucket_extraction_wb_data'
    table_id = 'tabla_transformada_prueba'
    table_full_id = f"{client.project}.{dataset_id}.{table_id}"

    # Obtén la información del archivo desde el bucket
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(file_name)
    file_contents = blob.download_as_text()

    # Parsea el contenido del archivo como JSON
    json_data = [line.split(',') for line in file_contents.split('\n')]
    field_names = json_data[0]
    json_data = [dict(zip(field_names, line)) for line in json_data[1:]]

    # Verifica si la tabla existe y, si no, la crea
    try:
        client.get_table(table_full_id)
    except Exception as e:
        if "Not found" in str(e):
            schema = [
                bigquery.SchemaField("Pais", "STRING"),
                bigquery.SchemaField("Anio", "INTEGER"),
                bigquery.SchemaField("Tasa_Natalidad", "FLOAT64"),
                bigquery.SchemaField("Emisiones_CO2", "FLOAT64"),
                bigquery.SchemaField("Educacion_obligatoria_en_anos", "FLOAT64"),
                bigquery.SchemaField("Tasa_Mortalidad", "FLOAT64"),
                bigquery.SchemaField("Gasto_Salud_Gobierno", "FLOAT64"),
                bigquery.SchemaField("PIB_per_capita", "FLOAT64"),
                bigquery.SchemaField("Esperanza_vida_femenina", "FLOAT64"),
                bigquery.SchemaField("Esperanza_vida_total", "FLOAT64"),
                bigquery.SchemaField("Tasa_alfabetizacion_adultos", "FLOAT64"),
                bigquery.SchemaField("Mortalidad_adulta_femenina", "FLOAT64"),
                bigquery.SchemaField("Mortalidad_adulta_masculina", "FLOAT64"),
                bigquery.SchemaField("Mortalidad_infantil", "FLOAT64"),
                bigquery.SchemaField("Acceso_agua_potable", "FLOAT64"),
                bigquery.SchemaField("Poblacion_rural", "FLOAT64"),
            ]
            table = bigquery.Table(table_full_id, schema=schema)
            client.create_table(table)

    # Carga los datos en BigQuery
    table_ref = client.dataset(dataset_id).table(table_id)
    table = client.get_table(table_ref)

    errors = client.insert_rows_json(table, json_data)

    if errors:
        return f'Error al cargar datos a BigQuery: {errors}'
    else:
        return 'Datos cargados exitosamente a BigQuery.'